<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/grdrive')

Mounted at /content/grdrive


In [ ]:
!pip3 install datasets==1.0.2
!pop3 install tqdm==4.55.1

In [4]:
!mkdir '/content/grdrive/My Drive/t5'
!mkdir '/content/grdrive/My Drive/t5/dataset'
!mkdir '/content/grdrive/My Drive/t5/model'
!mkdir '/content/grdrive/My Drive/t5/tokenizer'

In [5]:
from datasets import load_dataset
train_dataset = load_dataset('squad', split='train')
valid_dataset = load_dataset('squad', split='validation')

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/955 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41. Subsequent calls will reuse this data.


Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


In [7]:
# check
from pprint import pprint
sample_valid = next(iter(valid_dataset))

context = sample_valid['context']
question = sample_valid['question']
answer = sample_valid['answers']['text'][0]

print('context : ', context)
print('question : ', question)
print('answer : ', answer )

context :  Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
question :  Which NFL team represented the AFC at Super Bowl 50?
answer :  Denver Broncos


In [9]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

from tqdm.notebook import tqdm

df_train = pd.DataFrame(columns=['context', 'answer', 'question'])
df_valid = pd.DataFrame(columns=['context', 'answer', 'question'])

In [10]:
count_long = 0
count_short = 0

for index,val in enumerate(tqdm(train_dataset)):
  context = val['context']
  answer = val['answers']['text'][0]
  question = val['question']

  no_of_words = len(answer.split())

  if no_of_words >= 7:
    count_long += 1
    continue
  
  else:
    df_train.loc[count_short] = [context] + [answer] + [question]
    count_short += 1

print(count_long, count_short)

  0%|          | 0/87599 [00:00<?, ?it/s]

8935 78664


In [11]:
count_long = 0
count_short = 0

for index,val in enumerate(tqdm(valid_dataset)):
  context = val['context']
  answer = val['answers']['text'][0]
  question = val['question']

  no_of_words = len(answer.split())

  if no_of_words >= 7:
    count_long += 1
    continue
  
  else:
    df_valid.loc[count_short] = [context] + [answer] + [question]
    count_short += 1

print(count_long, count_short)

  0%|          | 0/10570 [00:00<?, ?it/s]

918 9652


In [12]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_valid = shuffle(df_valid)

In [13]:
train_save_path = '/content/grdrive/My Drive/t5/dataset/squad_t5_train.csv'
valid_save_path = '/content/grdrive/My Drive/t5/dataset/squad_t5_valid.csv'

df_train.to_csv(train_save_path, index=False)
df_valid.to_csv(valid_save_path, index=False)

In [14]:
####################################################

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install transformers==4.1.1
!pip3 install pytorch-lightning==1.1.3
!pip3 install tokenizers==0.9.4
!pip3 install sentencepiece==0.1.94
!pip3 install tqdm==4.56.0

In [ ]:
!pip install git+https://github.com/PyTorchLightning/lightning-bolts

In [ ]:
#restart runtime
import os 

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [1]:
train_save_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_train.csv'
valid_save_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_valid.csv'

In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from termcolor import colored
import textwrap

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

pl.seed_everything(42)

Global seed set to 42


42

In [3]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from tqdm.notebook import tqdm
import copy 

In [8]:
class QuestionGenerationDataset(Dataset):
  def __init__(self, tokenizer, filepath, max_in_len=512, max_out_len=96):
    self.path = filepath

    self.data = pd.read_csv(self.path, nrows=1000)###########################################

    self.max_len_in = max_in_len
    self.max_len_out = max_out_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []
    self.skipped_count = 0
    self._build()

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask = self.inputs[index]["attention_mask"].squeeze()
    target_mask = self.targets[index]["attention_mask"].squeeze()
    
    labels = copy.deepcopy(target_ids)
    labels[labels==0] = -100

    return {"source_ids" : source_ids, "source_mask" : src_mask, "target_ids " : target_ids, "target_mask" : target_mask, "labels" : labels}
  
  def _build(self):
    for idx in tqdm(range(len(self.data))):
      context, ans, target = self.data.loc[idx, "context"], self.data.loc[idx, "answer"], self.data.loc[idx, "question"] 


      input = "context: %s answer: %s </s>" % (context, ans)
      target = "question: %s </s>" % (str(target))

      test_input_encoding = self.tokenizer.encode_plus(
                              input,
                              truncation=False,
                              return_tensors='pt'
                              )
      
      len_in_str = len(test_input_encoding['input_ids'][0])

      if len_in_str > self.max_len_in:
        self.skipped_count += 1
        continue
      
      tokenized_ins = self.tokenizer.batch_encode_plus(
                              [input],
                              max_length=self.max_len_in,
                              pad_to_max_length=True,
                              return_tensors='pt'
                              )
      
      tokenized_targets = self.tokenizer.batch_encode_plus(
                              [target],
                              max_length=self.max_len_out,
                              pad_to_max_length=True,
                              return_tensors='pt'
                              )
      
      self.inputs.append(tokenized_ins)
      self.targets.append(tokenized_targets)

In [9]:
train_dataset = QuestionGenerationDataset(t5_tokenizer, train_save_path)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padd

In [10]:
valid_dataset = QuestionGenerationDataset(t5_tokenizer, valid_save_path)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams, t5model, t5tokenizer):
    super(T5FineTuner, self).__init__()
    self.hparams = hparams
    self.model = t5model
    self.tokenizer = t5tokenizer

  def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
    outputs = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=decoder_attention_mask,
        labels=lm_labels,
    )
    return outputs

  def training_step(self, batch, batch_idx):
    outputs = self.forward(
        input_ids=batch['source_ids'],
        attention_mask=batch['source_mask'],
        decoder_input_ids=batch['target_ids'],
        decoder_attention_mask=batch['target_mask'],
        lm_labels=batch['labels']
    )

    loss = outputs[0]
    self.log('train_loss',loss)
    return loss

  def validation_step(self, batch, batch_idx):
    outputs = self.forward(
        input_ids=batch['source_ids'],
        attention_mask=batch['source_mask'],
        decoder_input_ids=batch['target_ids'],
        decoder_attention_mask=batch['target_mask'],
        lm_labels=batch['labels']
    )

    loss = outputs[0]
    self.log('val_loss',loss)
    return loss

  def train_dataloader(self):
    return DataLoader(train_dataset, batch_size=self.hparams.batch_size, num_workers=4)

  def val_dataloader(self):
    return DataLoader(valid_dataset, batch_size=self.hparams.batch_size, num_workers=4)

  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
    return optimizer

In [ ]:
args_dict = dict(batch_size=4)
args = argparse.Namespace(**args_dict)

model = T5FineTuner(args, t5_model, t5_tokenizer)
trainer = pl.Trainer(max_epochs=1 ,gpus=1, progress_bar_refresh_rate=30)
trainer.fit(model)